# Thermodynamics Regression
## Background
This is specifc for the purple model built, with double island and foil on top of the lid.
## Newton's Law of Cooling: 

$$k = -\frac{1}{t}ln\frac{T_{s} - T_{r}}{T_{f} - T_{r}}$$

$$t: duration$$
$$T_{s}: starting\space temperature$$
$$T_{f}: final\space temperature$$
$$T_{r}: room\space temperature$$

## Functions

### The Following Functions Are Defined
* RSS
* MSE
* K-fold cross-validation
* Find best L2 penalty based on cross-validation
* Print linear regression
* Predict with the cofficients

In [1]:
import graphlab
import numpy as np

# convenient functions
squared = lambda x: x*x

def get_residual_sum_of_squares(model, data, outcome):
    return sum(map(squared, outcome - model.predict(data)))

def get_mean_residual_sum_of_squares(model, data, outcome):
    return sum(map(squared, outcome - model.predict(data)))/len(outcome)

def k_fold_cross_validation(k, l2_penalty, data, output_name, features_list):
    shuffled_data = graphlab.toolkits.cross_validation.shuffle(data, random_seed=1)
    n = len(shuffled_data)
    mse_average = 0
    
    for i in xrange(k):
        start = (n*i)/k
        end = (n*(i+1))/k-1
        validation = shuffled_data[start:end]
        train = shuffled_data[0:start].append(shuffled_data[end+1:n])
        model = graphlab.linear_regression.create(train, target = output_name, features = features_list, validation_set = None, l2_penalty=l2_penalty)
        
        mse_average = mse_average + get_mean_residual_sum_of_squares(model, data, data[output_name])
    
    return mse_average/k

def find_best_lambda(folds, input_data, output_name, feature_names):
    penalties = np.logspace(1, 7, num=13)
    lowest_validation_error = float("inf")
    best_lambda = 0
    
    for penalty in penalties:
        mse_average = k_fold_cross_validation(k=folds, l2_penalty=penalty, data=input_data, output_name = output_name, features_list = feature_names)

        if (mse_average < lowest_validation_error):
            lowest_validation_error = mse_average
            best_lambda = penalty

    return best_lambda

def print_regression(coefficients):
    #intercept
    presentation = "%8.6f" % coefficients['value'][0]
    
    for i in xrange(1, len(coefficients)):
        s = "(%8.6f)" % coefficients['value'][i]

        presentation = s + "x(" + coefficients['name'][i] + ") + " + presentation
        
    return presentation

def predict_any(rm_t, start_t, k_value, duration, cofficients):
    return start_t*cofficients['value'][2]+rm_t*cofficients['value'][1]+cofficients['value'][0]


## Setting Things UP
### Set up environment
* Read Files
* Split between training_validation and test sets.
* Set up N and K for cross-validation
* set up features for cross-validation

### The plan: 1. use k-value, T<sub>r</sub>, T<sub>s</sub>, and t to regress for T<sub>f</sub>

In [ ]:
# read files
purple_data_all = graphlab.SFrame.read_csv('purple.specific.data.all.csv')
purple_data_double_island = graphlab.SFrame.read_csv('purple.specific.data.double.island.csv')
purple_data_old = graphlab.SFrame.read_csv('purple.specific.data.old.model.csv')

# split data for cross validation
purple_train_validate_data_all, purple_test_data_all = purple_data_all.random_split(.9,seed=0)
purple_train_validate_data_double_island, purple_test_data_double_island = purple_data_double_island.random_split(.9,seed=0)
purple_train_validate_data_old, purple_test_data_old = purple_data_old.random_split(.9,seed=0)

# set up for "leave one out" cross validation
n_all = len(spurple_train_validate_data_all)
n_double_island = len(spurple_train_validate_data_double_island)
n_old = len(spurple_train_validate_data_old)
k_all = n_all
k_double_island = n_double_island
k_old = n_old

# set up features for cross-validation
# the plan: 1. use k-value, T_r, T_s, and t to regress for T_f
xthermo_features = ['Room temperature', 'Start temperature', 'K Value Calculator', 'Duration']
xoutput_feature = 'End temperature'

In [ ]:
best_lambda_all = find_best_lambda(k_all, purple_train_validate_data_all, xoutput_feature, xthermo_features)
best_lambda_double_island = find_best_lambda(k_double_island, purple_train_validate_data_double_island, xoutput_feature, xthermo_features)
best_lambda_old = find_best_lambda(k_old, purple_train_validate_data_old, xoutput_feature, xthermo_features)

In [ ]:
print 'lambda_all: ', best_lambda_all, 'lambda_double_island: ', best_lambda_double_island, 'lambda_old', best_lambda_old

## Build models with Best Lambda Identified

In [ ]:
model_all = graphlab.linear_regression.create(purple_train_validate_data_all, target = xoutput_feature, features = xthermo_features, validation_set = None, l2_penalty=best_lambda_all)
model_double_island = graphlab.linear_regression.create(purple_train_validate_data_double_island, target = xoutput_feature, features = xthermo_features, validation_set = None, l2_penalty=best_lambda_double_island)
model_old = graphlab.linear_regression.create(purple_train_validate_data_old, target = xoutput_feature, features = xthermo_features, validation_set = None, l2_penalty=best_lambda_old)

## Print Coefficients and MSE

In [ ]:
print print_regression(model_all['coefficients'])
print print_regression(model_double_island['coefficients'])
print print_regression(model_old['coefficients'])

mse_all = get_mean_residual_sum_of_squares(model_all, purple_test_data_all, purple_test_data_all[xoutput_feature])
mse_double_island = get_mean_residual_sum_of_squares(model_double_island, purple_test_data_double_island, purple_test_data_double_island[xoutput_feature])
mse_old = get_mean_residual_sum_of_squares(model_old, purple_test_data_old, purple_test_data_old[xoutput_feature])

print mse_all, " ", mse_double_island, " ", mse_old

## Use the mean of k value to make a prediction
Predict with room temperature 21, start temperature 71, duration 25, and mean k-value from each corresponding data set

In [ ]:
k_all_mean = np.mean(np.array(purple_data_all['K Value Calculator']))
k_double_island_mean = np.mean(np.array(purple_data_double_island['K Value Calculator']))
k_old_mean = np.mean(np.array(purple_data_old['K Value Calculator']))

v = predict_any(21, 71, k_double_island_mean, 25, model_double_island['coefficients'])

print "%5.2f" % v

v = predict_any(21, 71, k_all_mean, 25, model_all['coefficients'])

print "%5.2f" % v